# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [2]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_file

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [25]:
# set path where C files reside

path = r"linux-kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

['tsacct.c', 'Kconfig.kexec', 'latencytop.c', 'gcov', 'power', 'static_call_inline.c', 'Kconfig.hz', 'audit.h', 'iomem.c', 'kexec_core.c', 'kcsan', 'futex', 'kallsyms_internal.h', 'async.c', 'utsname.c', 'rseq.c', 'kallsyms_selftest.h', 'exec_domain.c', 'smpboot.c', 'bounds.c', 'trace', 'sys.c', 'compat.c', 'audit_watch.c', 'Kconfig.freezer', 'usermode_driver.c', 'resource.c', 'stop_machine.c', 'livepatch', 'audit_tree.c', 'cpu.c', 'delayacct.c', 'Kconfig.preempt', 'kcov.c', 'static_call.c', 'sysctl.c', 'ptrace.c', 'user.c', 'fork.c', 'backtracetest.c', 'entry', 'user_namespace.c', 'Makefile', 'kexec.c', 'regset.c', 'module', 'extable.c', 'irq', 'kexec_internal.h', 'locking', 'pid.c', 'padata.c', 'kexec_elf.c', 'fail_function.c', 'cfi.c', 'exit.c', 'panic.c', 'module_signature.c', 'kexec_file.c', 'up.c', 'resource_kunit.c', 'groups.c', 'vmcore_info.c', 'auditsc.c', 'workqueue_internal.h', 'hung_task.c', 'sched', 'ucount.c', 'cgroup', 'acct.c', 'tracepoint.c', 'reboot.c', 'uid16.h', 'se

In [26]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)
        
print(c_files)

['tsacct.c', 'latencytop.c', 'static_call_inline.c', 'iomem.c', 'kexec_core.c', 'async.c', 'utsname.c', 'rseq.c', 'exec_domain.c', 'smpboot.c', 'bounds.c', 'sys.c', 'compat.c', 'audit_watch.c', 'usermode_driver.c', 'resource.c', 'stop_machine.c', 'audit_tree.c', 'cpu.c', 'delayacct.c', 'kcov.c', 'static_call.c', 'sysctl.c', 'ptrace.c', 'user.c', 'fork.c', 'backtracetest.c', 'user_namespace.c', 'kexec.c', 'regset.c', 'extable.c', 'pid.c', 'padata.c', 'kexec_elf.c', 'fail_function.c', 'cfi.c', 'exit.c', 'panic.c', 'module_signature.c', 'kexec_file.c', 'up.c', 'resource_kunit.c', 'groups.c', 'vmcore_info.c', 'auditsc.c', 'hung_task.c', 'ucount.c', 'acct.c', 'tracepoint.c', 'reboot.c', 'seccomp.c', 'kcmp.c', 'kheaders.c', 'crash_core.c', 'watch_queue.c', 'range.c', 'watchdog.c', 'dma.c', 'vhost_task.c', 'softirq.c', 'stackleak.c', 'audit.c', 'scftorture.c', 'notifier.c', 'user-return-notifier.c', 'jump_label.c', 'irq_work.c', 'freezer.c', 'elfcorehdr.c', 'utsname_sysctl.c', 'kthread.c', 'p

In [27]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [28]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0
#define pr_fmt(fmt) "kcov: " fmt

#define DISABLE_BRANCH_PROFILING
#include <linux/atomic.h>
#include <linux/compiler.h>
#include <linux/errno.h>
#include <linux/export.h>
#include <linux/types.h>
#include <linux/file.h>
#include <linux/fs.h>
#include <linux/hashtable.h>
#include <linux/init.h>
#include <linux/jiffies.h>
#include <linux/kmsan-checks.h>
#include <linux/mm.h>
#include <linux/preempt.h>
#include <linux/printk.h>
#include <linux/sched.h>
#include <linux/slab.h>
#include <linux/spinlock.h>
#include <linux/vmalloc.h>
#include <linux/debugfs.h>
#include <linux/uaccess.h>
#include <linux/kcov.h>
#include <linux/refcount.h>
#include <linux/log2.h>
#include <asm/setup.h>

#define kcov_debug(fmt, ...) pr_debug("%s: " fmt, __func__, ##__VA_ARGS__)

/* Number of 64-bit words written per one comparison: */
#define KCOV_WORDS_PER_CMP 4

/*
 * kcov descriptor (one per opened debugfs file).
 * State transitions of the descriptor:
 *  - initial state 

In [29]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 2228669


In [30]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [31]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [32]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 98


## Divide data in input (X) and output (y)

### Create sequences

In [55]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [56]:
print("Number of training samples: {}".format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [57]:
# create X and Y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=np.bool_)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [58]:
print("Shape of X: {}".format(X.shape))
print("Shape of Y: {}".format(y.shape))

Shape of X: (133317, 50, 98)
Shape of Y: (133317, 98)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [62]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=0.1))
model.add(LSTM(128, dropout=0.1))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [63]:
# check model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 50, 128)           116224    
                                                                 
 lstm_15 (LSTM)              (None, 128)               131584    
                                                                 
 dense_7 (Dense)             (None, 98)                12642     
                                                                 
Total params: 260450 (1017.38 KB)
Trainable params: 260450 (1017.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1042/1042 [==============================] - 149s 142ms/step - loss: 3.1747 - acc: 0.1865
Epoch 2/20
1042/1042 [==============================] - 149s 143ms/step - loss: 2.5657 - acc: 0.3135
Epoch 3/20
1042/1042 [==============================] - 159s 152ms/step - loss: 2.3349 - acc: 0.3702
Epoch 4/20
1042/1042 [==============================] - 156s 150ms/step - loss: 2.1665 - acc: 0.4141
Epoch 5/20
1042/1042 [==============================] - 154s 147ms/step - loss: 2.0413 - acc: 0.4442
Epoch 6/20
1042/1042 [==============================] - 154s 148ms/step - loss: 1.9360 - acc: 0.4716
Epoch 7/20
1042/1042 [==============================] - 154s 148ms/step - loss: 1.8489 - acc: 0.4927
Epoch 8/20
1042/1042 [==============================] - 148s 142ms/step - loss: 1.7780 - acc: 0.5111
Epoch 9/20
1042/1042 [==============================] - 151s 144ms/step - loss: 1.7130 - acc: 0.5280
Epoch 10/20
1042/1042 [==============================] - 150s 144ms/step - loss: 1.6610 - a

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [65]:
# define function to sample next word from a probability array based on temprature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [66]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[ 0, 10,  0],
       [ 0,  8,  2]])

In [68]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
    print('-'*50, 'diversity:', diversity)
    
    generated = ''
    sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
    generated += sentence
    print('------ Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(1000):
        x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
------ Generating with seed: " *   between [1] and [3]:
 *
 *       [abort_ip]
 "
 *   between [1] and [3]:
 *
 *       [abort_ip]
 * @resource start struct static_call_site *static_call_site_mstime,
				   struct resource *res);
	int retval;
	struct audit_tree *current;
	struct resource *res;

	if (!new->start = 0)
		return -EINVAL;

	return true;

	if (!pur == int) && SMIT_TERE_ONLINE;
		return -ENOMEM;
	return -EINVAL;

	return cpuhp_device_state(cpu, state, cpu, state);
}

/t *in inter mstine in the stop_machine filed that it is someroniziging
 * @dev: CPU is stop_wn_cpu_state inits any inserting this stop_add_fn_t fild
 * @fn_t a down state static struct static_call_init(struct resource *res);
	int retval;
	int ret;

	return ret;
}

static int cpuhp_init_cpus_stop_exed_cpu_desor(struct cpuhp_step_state, bringup)
{
	struct audit_tree *tre;
	int ret;

	return call_resource(rem);
	return -EFAULT;

	if (!caseblock() {
	

In [69]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "		list_del_init(&rule->rlist);
		if (rule->tree) {"
		list_del_init(&rule->rlist);
		if (rule->tree) {
			return -EINVAL;
		}
			list_del_get(&fagst);
		break;
		}

		return -EINVAL;
	}

	return kegid_region(pexed_inode_map);

	/*
	 * compation. */
	krule_unlock();
	if (!cpuhp_stop_et_cpu()))
		return -EINVAL;
	return cpuhp_init_init(cpu);
	return cpuhp_wask_cpu(state, 0, CPUHP_AP_ONLINE);
}

/*
 * In intert.. */
static int cpuhp_stop_cpu_stop_cpu_state static_call_sites,
					           unsigned long astr,
			          urrent)
{
	struct resource *res;
	int ret;

	return cpu_pic_tree_cpu();
	return = res->start = force;

	return key;
}

static int cpuhp_stop_cpus_read_state()
	struct resource *res;
	struct resource *res;
	struct resource *conflict;

	if (!audit_watch_chunk);
		return -EINVAL;
	}

	/*
	 * We completed a date is is callback to set all initialized and static instructions used be